In [2]:
import hax

/home/erik/anaconda3/envs/pax/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [3]:
class ExtraS1S2Properties(hax.minitrees.TreeMaker):
    '''
    
    '''
    __version__ = '0.1.4' # beta
    extra_branches = ['peaks.*', 'interactions*']
    
    def remove_duplicates_from_list(self, seq):
        seen = set()
        seen_add = seen.add
        return [x for x in seq if not (x in seen or seen_add(x))]
    
    
    def extract_data(self, event):
        # ================ PEAK PROPERTIES =========================
        # Extract data for these types
        peak_types = ['s1', 's2']
        
        # Peak_properties (will be loaded for largest s1 and s2)
        peak_properties = ['n_saturated_channels', 'center_time', 'left']
        # Peak properties for S1 and S2 that are in lists...
        extra_properties = ['time_from_midpoint_10p', 'time_from_midpoint_20p',
                            'time_from_midpoint_30p', 'time_from_midpoint_40p']
        largest_other_properties = ['area_fraction_top', 'center_time']

        result_vars = [_pt + '_' + _pp for _pt in peak_types for _pp in peak_properties ]
        result = {k: float('nan') for k in result_vars}
        result['largest_other_s1_area_fraction_top'] = float(-1)
        result['largest_other_s2_area_fraction_top'] = float(-1)
        
        
        s1s = []
        s2s = []
        n_interactions = len(event.interactions)
        
        # For all interactions, take their s1 and s2 index
        for interaction in event.interactions:
            # Get s1 and s2 index
            s1s.append(interaction.s1)
            s2s.append(interaction.s2)

        
        # ================= LOAD PEAK DATA ==========================
        # Loop over the two peak types
        for pt in peak_types:
            # Load index list
            if pt == 's1':
                # Peak list: list ordered by interaction
                peak_list = s1s
                # Peak list all: all peaks labeled as S1
                peak_list_all = sorted(event.s1s, key = lambda index: - getattr(event.peaks[index], 'area'))

            elif pt == 's2':
                peak_list = s2s
                peak_list_all = sorted(event.s2s, key = lambda index: - getattr(event.peaks[index], 'area'))
            else:
                raise NotImplementedError('Peak types s1 and s2 supported only!')
            
            # Now, for this peak type, loop over peak_indices and get peak pos
            if len(peak_list) == 0:
                # If there are not enough peaks of this type, do not try to find properties
                continue
            else:
                
                # Main S1 or S2
                peak_pos = int(peak_list[0])
                peak_name = pt + '_' # Example: s1_
                # All standard properties
                for prop in peak_properties:
                    result[peak_name + prop] = getattr(event.peaks[peak_pos], prop)
                    
                # Here come nonstandard properties
                for i in range(10):
                    if ('range_%d0p_area' % i) in extra_properties:
                        result[peak_name + 'range_%d0p_area' % i] = (
                            list(event.peaks[peak_pos].range_area_decile)[i])
                    if ('time_from_midpoint_%d0p' % i) in extra_properties:
                        result[peak_name + 'time_from_midpoint_%d0p' % i] = (
                            list(event.peaks[peak_pos].area_decile_from_midpoint)[i])
                
                
                # Remove the main S1/S2 from list
                peak_list_all.remove(peak_pos)
                if len(peak_list_all) > 0:
                    largest_other_index = peak_list_all[0]
                    for prop in largest_other_properties:
                        result['largest_other_' + pt + '_' + prop] = getattr(event.peaks[largest_other_index], prop)
                
        
        return result

In [ ]:
class XAMSProperties(hax.minitrees.TreeMaker):
    '''
    Get properties for XAMS
    
    '''
    __version__ = '0.1.0'
    
    extra_branches = ['peaks.*', 'peaks.range_area_decile*', 
                      'interactions*']
    
    def extract_data(self, event):
        
        peak_types = ['s1', 's2']
        peak_properties = ['area_fraction_top', 'area', 'center_time', 'n_saturated_channels']
        extra_properties = ['range_50p_area', 'range_70p_area']
        all_properties = peak_properties + extra_properties
        result_vars = [_pt + '_' + _pp for _pt in peak_types for _pp in all_properties ]
        result = {k: float('nan') for k in result_vars}
        
        interaction_properties = ['drift_time']
        for _prop in interaction_properties:
            result[_prop] = float('nan')
            
        s1s = []
        s2s = []
        
        for peak in event.peaks:
            if peak.type == 's1':
                s1s.append(peak)
            if peak.type == 's2':
                s2s.append(peak)
        
        # Reverse-sort by area
        s1s = sorted(s1s, key = lambda peak: - peak.area)
        s2s = sorted(s2s, key = lambda peak: - peak.area)

        
        # Now load properies
        result['n_s1s'] = len(event.s1s)
        result['n_s2s'] = len(event.s2s)
        
        
        for peak_name, peak_list in zip(['s1', 's2'], [s1s, s2s]):
            if len(peak_list) == 0:
                continue
            # First peak in list sorted by area
            peak = peak_list[0]
            for _prop in peak_properties:
                result[peak_name + '_' + _prop] = getattr(peak, _prop)
            for i in range(10):
                if ('range_%d0p_area' % i) in extra_properties:
                    result[peak_name + '_' + 'range_%d0p_area' % i] = list(peak.range_area_decile)[i]
        
        if len(s2s) > 1:
            result['largest_other_s2'] = getattr(s2s[1], 'area')
        else:
            result['largest_other_s2'] = 0.
            
        result['drift_time'] = result['s2_center_time'] - result['s1_center_time']
        



#             for peak_type, peak_index in zip(peak_types, [s1,s2]):
#                 for _prop in peak_properties:
#                     result[peak_name + '_' + _prop] = getattr(event.peaks[peak_index], _prop)

#                 if 'range_50p_area' in extra_properties:
#                     result[peak_name + '_' + 'range_50p_area'] = list(event.peaks[peak_pos].range_area_decile)[5]
        
        return result

In [ ]:
class NaIProperties(hax.minitrees.TreeMaker):
    '''
    Add the properties that are closest to the TPC interaction.
    Proximity based on a small delay (since pulse comes a bit later)
    '''
    __version__ = '0.1.3'
    extra_branches = ['peaks.*', 'interactions*']
    s1_offset = 16. # ns from left edge
    
    def extract_data(self, event):
        ret = {
            'NaI_area' : float('nan'),
            'NaI_time_from_s1': float('nan'),
            'NaI_time_from_s1_corr': float('nan'),
            'NaI_left' : float('nan'),
            'NaI_n_peaks' : 0,
              }
        # Check if the interaction exists
        if len(event.interactions) == 0:
            return ret
        
        main_s1_index = event.interactions[0].s1
        s1_left = getattr(event.peaks[main_s1_index], 'left')        
        
        time_from_s1 = []
        time_from_s1_corr = []
        area = []
        left = []
        
        for p in event.peaks:
            if getattr(p, 'detector') == 'nai':
                time_from_s1.append((getattr(p, 'left') - s1_left) * 2.)
                time_from_s1_corr.append((getattr(p, 'left') - s1_left) * 2. - self.s1_offset)
                area.append(getattr(p, 'area'))
                left.append(getattr(p, 'left'))
        
        if len(area) == 0:
            return ret
        
        index_of_closest_peak = np.argmin(abs(np.array(time_from_s1_corr)))
        ret['NaI_area'] = area[index_of_closest_peak]
        ret['NaI_left'] = left[index_of_closest_peak]
        ret['NaI_time_from_s1'] = time_from_s1[index_of_closest_peak]
        ret['NaI_time_from_s1_corr'] = time_from_s1_corr[index_of_closest_peak]
        ret['NaI_n_peaks'] = len(area)
        return ret